In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [2]:
pip install tflearn

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import tflearn
import tensorflow as tf
import random

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [10]:
import json
with open('intents1.json') as json_data:
    intents = json.load(json_data)

In [12]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\basu9\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [13]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

32 documents
10 classes ['Booking', 'Checkin', 'Room Types', 'goodbye', 'greeting', 'opentoday', 'payments', 'rental', 'thanks', 'today']
62 unique stemmed words ["'d", "'s", 'a', 'acceiv', 'anyon', 'ar', 'at', 'book', 'bye', 'can', 'card', 'cash', 'check', 'checkin', 'credit', 'day', 'do', 'doe', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hotel', 'hour', 'how', 'i', 'is', 'kind', 'lat', 'lik', 'mastercard', 'of', 'on', 'op', 'plac', 'reach', 'rent', 'reserv', 'room', 'see', 'should', 'stay', 'tak', 'thank', 'that', 'the', 'ther', 'thi', 'tim', 'to', 'today', 'typ', 'want', 'we', 'what', 'when', 'which', 'work', 'yo', 'you']


In [14]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

C:\Users\basu9\AppData\Local\Temp\ipykernel_8808\1193693976.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [17]:
# reset underlying graph data
tf.compat.v1.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.95437 | time: 0.014s
| Adam | epoch: 1000 | loss: 0.95437 - acc: 0.8507 -- iter: 24/32
Training Step: 4000  | total loss: 0.87008 | time: 0.019s
| Adam | epoch: 1000 | loss: 0.87008 - acc: 0.8657 -- iter: 32/32
--
INFO:tensorflow:C:\Users\basu9\Desktop\Chatbot_hotelmanagement\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:C:\Users\basu9\Desktop\Chatbot_hotelmanagement\model.tflearn.data-00000-of-00001
INFO:tensorflow:0
INFO:tensorflow:C:\Users\basu9\Desktop\Chatbot_hotelmanagement\model.tflearn.index
INFO:tensorflow:0
INFO:tensorflow:C:\Users\basu9\Desktop\Chatbot_hotelmanagement\model.tflearn.meta
INFO:tensorflow:100


In [18]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [19]:
p = bow("is your shop open today?", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0]
['Booking', 'Checkin', 'Room Types', 'goodbye', 'greeting', 'opentoday', 'payments', 'rental', 'thanks', 'today']


In [20]:
print(model.predict([p]))

[[4.31924476e-04 1.14490412e-01 3.66954482e-04 1.43539505e-02
  2.22275108e-01 5.32072783e-01 3.60226748e-03 1.75733931e-05
  2.77860626e-03 1.09610394e-01]]


In [21]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )